# Imports

In [ ]:
import os, sys
import yfinance as yf
# import ta
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import date, timedelta, datetime
from itertools import product
from IPython.display import clear_output
import matplotlib.pyplot as plt
from csv import writer
import warnings
# import talib as ta
import math
import re
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import pandas as pd
import numpy as np
from typing import List, Tuple
import random
import csv
import numpy as np
import pandas as pd




# Trading Strategy implenetation class

In [ ]:
class TradingStrategy:
  def __init__(self, data: pd.DataFrame, initial_capital: float = 20000,
               n_positions: int = 20, stop_loss_pct: float = 0.05):
      self.data = data
      self.initial_capital = initial_capital
      self.current_capital = initial_capital
      self.n_positions = n_positions
      self.stop_loss_pct = stop_loss_pct
      self.portfolio_values = []
      self.trade_history = []

  def calculate_signals(self, start_idx: int) -> List[str]:
      """Calculate EMA signals for all stocks"""
      # Make sure we have enough data for EMA calculation
      if start_idx < 50:  # Need at least 50 days for EMA50
          return []

      # Calculate EMAs using data up to start_idx
      data_slice = self.data.iloc[max(0, start_idx-50):start_idx+1]
      ema10 = data_slice.ewm(span=10, adjust=False).mean()
      ema50 = data_slice.ewm(span=50, adjust=False).mean()

      # Compare EMAs at the current point
      buy_signals = []
      for col in self.data.columns:
          if ema10.iloc[-1][col] > ema50.iloc[-1][col]:
              buy_signals.append(col)

      return buy_signals

  def execute_trades(self, start_date_idx: int, holding_period: int) -> Tuple[float, List]:
      """Execute trades for one period"""
      buy_signals = self.calculate_signals(start_date_idx)

      if not buy_signals:  # If no signals, return immediately
          return start_date_idx, []

      if len(buy_signals) < self.n_positions:
          selected_stocks = buy_signals
      else:
          selected_stocks = random.sample(buy_signals, self.n_positions)

      position_size = self.current_capital / len(selected_stocks)
      positions = []

      for stock in selected_stocks:
          entry_price = self.data.iloc[start_date_idx][stock]
          stop_loss = entry_price * (1 - self.stop_loss_pct)
          shares = position_size / entry_price
          positions.append({
              'stock': stock,
              'shares': shares,
              'entry_price': entry_price,
              'stop_loss': stop_loss
          })

      period_values = []
      closed_positions = []

      end_idx = min(start_date_idx + holding_period, len(self.data))

      for day in range(start_date_idx + 1, end_idx):
          daily_value = 0

          for pos in positions[:]:
              current_price = self.data.iloc[day][pos['stock']]

              if current_price <= pos['stop_loss']:
                  value = pos['shares'] * current_price
                  daily_value += value
                  closed_positions.append({
                      'stock': pos['stock'],
                      'entry_price': pos['entry_price'],
                      'exit_price': current_price,
                      'pnl': (current_price - pos['entry_price']) * pos['shares']
                  })
                  positions.remove(pos)
              else:
                  daily_value += pos['shares'] * current_price

          if daily_value > 0:  # Only append if we have active positions
              period_values.append(daily_value)

          if not positions:  # All positions closed
              break

      # Close remaining positions at end of period
      final_day = min(start_date_idx + holding_period - 1, len(self.data) - 1)
      for pos in positions:
          exit_price = self.data.iloc[final_day][pos['stock']]
          closed_positions.append({
              'stock': pos['stock'],
              'entry_price': pos['entry_price'],
              'exit_price': exit_price,
              'pnl': (exit_price - pos['entry_price']) * pos['shares']
          })

      # Update capital
      total_pnl = sum(pos['pnl'] for pos in closed_positions)
      self.current_capital += total_pnl

      if period_values:
          self.portfolio_values.extend(period_values)

      return final_day, closed_positions

  def run_strategy(self, start_idx: int, n_periods: int, holding_period: int) -> dict:
      """Run the complete trading strategy"""
      self.current_capital = self.initial_capital
      self.portfolio_values = []
      current_idx = start_idx

      for _ in range(n_periods):
          if current_idx >= len(self.data) - holding_period:
              break
          next_idx, trades = self.execute_trades(current_idx, holding_period)
          current_idx = next_idx + 1
          self.trade_history.extend(trades)

      # Calculate metrics
      if not self.portfolio_values:
          return {
              'PNL': self.current_capital - self.initial_capital,
              'CAGR': 0,
              'Max_Drawdown': 0,
              'CALMAR': 0
          }

      portfolio_values = np.array(self.portfolio_values)
      total_days = len(portfolio_values)

      # Calculate metrics
      pnl = self.current_capital - self.initial_capital

      # Calculate CAGR
      years = total_days / 252  # Assuming 252 trading days per year
      cagr = (self.current_capital / self.initial_capital) ** (1/years) - 1 if years > 0 else 0

      # Calculate Max Drawdown
      peak = portfolio_values[0]
      max_drawdown = 0

      for value in portfolio_values:
          if value > peak:
              peak = value
          drawdown = (peak - value) / peak
          max_drawdown = max(max_drawdown, drawdown)

      # Calculate CALMAR
      calmar = abs(cagr / max_drawdown) if max_drawdown != 0 else 0

      return {
          'PNL': pnl,
          'CAGR': cagr,
          'Max_Drawdown': max_drawdown,
          'CALMAR': calmar
      }

# Data Loading

In [ ]:
from data_loader2 import SNP100DataLoader

# Initialize the loader
loader = SNP100DataLoader()


# Load data for all stocks
df = loader.get_data(1298, include_indices=False)  # Last trading days from Nov 14, 2023


print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"Columns included: {df.columns.tolist()}")

Date range: 2019-11-26 00:00:00 to 2024-11-14 00:00:00
Columns included: ['NVDA', 'AAPL', 'MSFT', 'AMZN', 'GOOGL', 'GOOG', 'META', 'TSLA', 'AVGO', 'COST', 'NFLX', 'ASML', 'TMUS', 'AMD', 'CSCO', 'PEP', 'ADBE', 'LIN', 'AZN', 'TXN', 'QCOM', 'INTU', 'ISRG', 'AMGN', 'CMCSA', 'PDD', 'BKNG', 'AMAT', 'HON', 'VRTX', 'PANW', 'ADP', 'MU', 'GILD', 'ADI', 'SBUX', 'MELI', 'INTC', 'LRCX', 'KLAC', 'MDLZ', 'REGN', 'CTAS', 'SNPS', 'CDNS', 'PYPL', 'CRWD', 'MRVL', 'MAR', 'CSX', 'ORLY', 'WDAY', 'CHTR', 'ADSK', 'FTNT', 'TTD', 'ROP', 'PCAR', 'NXPI', 'TEAM', 'FANG', 'MNST', 'CPRT', 'PAYX', 'AEP', 'ODFL', 'ROST', 'FAST', 'KDP', 'DDOG', 'EA', 'BKR', 'KHC', 'MCHP', 'VRSK', 'CTSH', 'LULU', 'EXC', 'XEL', 'CCEP', 'IDXX', 'ON', 'CSGP', 'ZS', 'TTWO', 'ANSS', 'CDW', 'DXCM', 'BIIB', 'ILMN', 'MDB', 'WBD', 'MRNA', 'DLTR', 'WBA']


In [ ]:
df

,NVDA,AAPL,MSFT,AMZN,GOOGL,GOOG,META,TSLA,AVGO,COST,...,ANSS,CDW,DXCM,BIIB,ILMN,MDB,WBD,MRNA,DLTR,WBA
Date,,,,,,,,,,,,,,,,,,,,,
2019-11-26,5.398740,64.056145,145.705521,89.847000,65.487846,65.516602,198.372726,21.927999,27.252428,276.848633,...,252.479996,129.764740,56.017502,302.989990,310.710114,149.630005,33.360001,20.330000,95.260002,46.714996
2019-11-27,5.433598,64.916542,145.983475,90.925499,65.444443,65.488670,201.393631,22.086000,27.439409,276.407593,...,255.449997,129.253235,56.882500,303.350006,312.198456,149.770004,33.119999,20.370001,93.029999,47.091038
2019-11-29,5.396252,64.773537,145.082626,90.040001,65.043434,65.088158,201.034714,21.996000,27.247253,275.534454,...,254.690002,127.927063,56.827499,299.809998,312.023346,148.699997,32.939999,20.360001,91.459999,46.691486
2019-12-02,5.209771,64.024635,143.328751,89.080002,64.283829,64.337997,199.100525,22.324667,26.774193,273.310425,...,250.699997,127.008247,55.980000,292.390015,310.963043,137.429993,32.740002,19.760000,89.470001,46.432972
2019-12-03,5.170185,62.883060,143.098679,88.498001,64.577095,64.605339,198.223175,22.413334,26.412287,271.729706,...,250.369995,126.117775,57.165001,290.700012,314.173157,142.679993,32.130001,21.280001,88.650002,46.213615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-08,147.630005,226.960007,422.540009,208.179993,178.350006,179.860001,589.340027,321.220001,183.639999,943.799988,...,342.179993,199.869995,69.959999,173.039993,151.089996,290.040009,9.180000,46.830002,61.410000,9.070000
2024-11-11,145.259995,224.229996,418.010010,206.839996,180.350006,181.970001,583.169983,350.000000,178.910004,932.880005,...,346.329987,197.809998,70.309998,172.029999,151.500000,291.570007,9.330000,42.750000,61.209999,9.390000
2024-11-12,148.289993,224.229996,423.029999,208.910004,181.619995,183.320007,584.820007,328.489990,176.220001,932.380005,...,344.480011,191.910004,74.339996,168.279999,147.619995,291.649994,9.220000,43.470001,62.270000,9.010000


In [ ]:
df.index[780]

Timestamp('2023-01-03 00:00:00')

# Iterative reinvesting Results generation

In [ ]:
def run_strategy_multiple_times(strategy, start_idx, n_periods, holding_period, num_runs=8):
    pnl_list = []
    cagr_list = []
    max_drawdown_list = []
    calmar_list = []

    for _ in range(num_runs):
        results = strategy.run_strategy(
            start_idx=start_idx,
            n_periods=n_periods,
            holding_period=holding_period
        )
        pnl_list.append(results['PNL'])
        cagr_list.append(results['CAGR'])
        max_drawdown_list.append(results['Max_Drawdown'])
        calmar_list.append(results['CALMAR'])

    mean_pnl = np.mean(pnl_list)
    mean_cagr = np.mean(cagr_list)
    mean_max_drawdown = np.mean(max_drawdown_list)
    mean_calmar = np.mean(calmar_list)

    return mean_pnl, mean_cagr, mean_max_drawdown, mean_calmar

def run_strategy_with_intervals(strategy, initial_start_idx, interval, n_periods, holding_period, nn=8, num_runs=8):
    results_list = []

    for i in range(nn):
        start_idx = initial_start_idx + i * interval
        mean_pnl, mean_cagr, mean_max_drawdown, mean_calmar = run_strategy_multiple_times(
            strategy, start_idx, n_periods, holding_period, num_runs
        )
        start_date = df.index[start_idx]
        end_date = df.index[start_idx + n_periods * holding_period]
        results_list.append([start_date, end_date, mean_pnl, mean_cagr, mean_max_drawdown, mean_calmar])

    # Output the consolidated mean with start date and end date as one row in a CSV
    with open('strategy_results.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Start Date', 'End Date', 'Mean PNL', 'Mean CAGR', 'Mean Max Drawdown', 'Mean CALMAR Ratio'])
        for result in results_list:
            writer.writerow([result[0], result[1], f"${result[2]:.2f}", f"{result[3]:.2%}", f"{result[4]:.2%}", f"{result[5]:.2f}"])

    for result in results_list:
        print(f"Start Date: {result[0]}")
        print(f"End Date: {result[1]}")
        print(f"Mean PNL: ${result[2]:.2f}")
        print(f"Mean CAGR: {result[3]:.2%}")
        print(f"Mean Max Drawdown: {result[4]:.2%}")
        print(f"Mean CALMAR Ratio: {result[5]:.2f}")
        print()


In [ ]:
strategy = TradingStrategy(df)
run_strategy_with_intervals(strategy, initial_start_idx=780, interval=30, n_periods=4, holding_period=60)

Start Date: 2023-01-03 00:00:00
End Date: 2023-12-15 00:00:00
Mean PNL: $3144.87
Mean CAGR: 16.88%
Mean Max Drawdown: 60.78%
Mean CALMAR Ratio: 0.28

Start Date: 2023-02-15 00:00:00
End Date: 2024-01-31 00:00:00
Mean PNL: $3118.78
Mean CAGR: 17.18%
Mean Max Drawdown: 87.91%
Mean CALMAR Ratio: 0.20

Start Date: 2023-03-30 00:00:00
End Date: 2024-03-14 00:00:00
Mean PNL: $3254.71
Mean CAGR: 17.48%
Mean Max Drawdown: 63.13%
Mean CALMAR Ratio: 0.29

Start Date: 2023-05-12 00:00:00
End Date: 2024-04-26 00:00:00
Mean PNL: $3693.61
Mean CAGR: 19.84%
Mean Max Drawdown: 70.24%
Mean CALMAR Ratio: 0.29

Start Date: 2023-06-27 00:00:00
End Date: 2024-06-10 00:00:00
Mean PNL: $1757.33
Mean CAGR: 9.42%
Mean Max Drawdown: 70.98%
Mean CALMAR Ratio: 0.14

Start Date: 2023-08-09 00:00:00
End Date: 2024-07-24 00:00:00
Mean PNL: $2475.99
Mean CAGR: 13.27%
Mean Max Drawdown: 69.48%
Mean CALMAR Ratio: 0.19

Start Date: 2023-09-21 00:00:00
End Date: 2024-09-05 00:00:00
Mean PNL: $2217.05
Mean CAGR: 11.89%
Me

 RUnning one start date iteration

In [ ]:
# strategy = TradingStrategy(df)
# results = strategy.run_strategy(
#   start_idx=990,  # Starting index (need at least 50 days for EMA50)
#   n_periods=4,  # Number of periods
#   holding_period=60  # Days to hold positions
# )

# print(f"PNL: ${results['PNL']:.2f}")
# print(f"CAGR: {results['CAGR']:.2%}")
# print(f"Max Drawdown: {results['Max_Drawdown']:.2%}")
# print(f"CALMAR Ratio: {results['CALMAR']:.2f}")

PNL: $3948.06
CAGR: 21.21%
Max Drawdown: 56.46%
CALMAR Ratio: 0.38
